Census of Drug and Alcohol Treatment Services in Northern Ireland

In [1]:
from gssutils import *
scraper = Scraper('https://www.health-ni.gov.uk/publications/census-drug-and-alcohol-treatment-services-northern-ireland-2017')
scraper

## Census of drug and alcohol treatment services in Northern Ireland 2017

### Distributions

1. Drug and Alcohol Census 2017 ([application/pdf](https://www.health-ni.gov.uk/sites/default/files/publications/health/drug-alcohol-census-2017.pdf))
1. Data from Census of Drug and Alcohol Treatment Services ([MS Excel Spreadsheet](https://www.health-ni.gov.uk/sites/default/files/publications/dhssps/data-census-drug-alcohol-treatment-services.xlsx))
1. Pre-release Access List Drug and Alcohol Census ([application/pdf](https://www.health-ni.gov.uk/sites/default/files/publications/dhssps/pre-release-drug-alcohol-census.pdf))


In [2]:
tabs = {tab.name: tab for tab in scraper.distribution(
    title='Data from Census of Drug and Alcohol Treatment Services').as_databaker()}
tabs.keys()

dict_keys(['Contents & Notes to tables', 'Table 1', 'Table 2', 'Table 3', 'Table 4', 'Table 5', 'Metadata'])

In [3]:
tables = []
for tab_name, script in [
    ('Table 1', 'Treatment Services by Age and Gender.ipynb'),
    ('Table 2', 'Treatment Services by Service Type(Age).ipynb'),
    ('Table 2', 'Treatment Services by Service Type(sex).ipynb'),
    ('Table 3', 'Treatment Services by Residential status(Age).ipynb'),
    ('Table 3', 'Treatment Services by Residential Status(sex).ipynb'),
    ('Table 4', 'Treatment Services by Trust(Age).ipynb'),
    ('Table 4', 'Treatment Services by Trust(sex).ipynb'),
    ('Table 5', 'Treatment Services by Comparison table.ipynb')
]:
    tab = tabs[tab_name]
    %run "$script"
    tables.append(new_table)

In [4]:
tidy = pd.concat(tables)
tidy.count()

Period                          724
Sex                             724
Age                             724
Service Type                    724
Residential Status              724
Treatment Type                  724
Health and Social Care Trust    724
Measure Type                    724
Unit                            724
Value                           724
dtype: int64

In [5]:
tidy['Treatment Type'].fillna('Total', inplace = True)

In [6]:
tidy.count()

Period                          724
Sex                             724
Age                             724
Service Type                    724
Residential Status              724
Treatment Type                  724
Health and Social Care Trust    724
Measure Type                    724
Unit                            724
Value                           724
dtype: int64

In [7]:
tidy['Treatment Type'].unique()

array(['Alcohol Only', 'Drugs Only', 'Drugs & Alcohol', 'Total', 'All',
       'Under 18s', '18 and over', 'Male', 'Female  ', 'Under 18 ',
       'Statutory', 'Non-statutory', 'Prison', 'Residential',
       'Non-residential / Mixed', 'Belfast', 'Northern', 'South Eastern',
       'Southern', 'Western', 'Emergency admissions (HIS)'], dtype=object)

In [8]:
tidy.head()

,Period,Sex,Age,Service Type,Residential Status,Treatment Type,Health and Social Care Trust,Measure Type,Unit,Value
0,1 March 2017,Persons,Under 18,All,All,Alcohol Only,All,Count,People,95
1,1 March 2017,Persons,Under 18,All,All,Drugs Only,All,Count,People,324
2,1 March 2017,Persons,Under 18,All,All,Drugs & Alcohol,All,Count,People,294
3,1 March 2017,Persons,Under 18,All,All,Total,All,Count,People,713
4,1 March 2017,Persons,18 and over,All,All,Alcohol Only,All,Count,People,2482


In [9]:
tidy = tidy[tidy['Treatment Type'] == 'Alcohol Only']

In [10]:
tidy.head()

,Period,Sex,Age,Service Type,Residential Status,Treatment Type,Health and Social Care Trust,Measure Type,Unit,Value
0,1 March 2017,Persons,Under 18,All,All,Alcohol Only,All,Count,People,95
4,1 March 2017,Persons,18 and over,All,All,Alcohol Only,All,Count,People,2482
8,1 March 2017,Persons,All years,All,All,Alcohol Only,All,Count,People,2577
12,1 March 2017,Male,Under 18,All,All,Alcohol Only,All,Count,People,31
16,1 March 2017,Male,18 and over,All,All,Alcohol Only,All,Count,People,1536


In [11]:
tidy['Sex'].unique()

array(['Persons', 'Male', 'Female', 'Male (%)', 'Female (%)',
       '% of Total', '% of all Males ', '% of all Females', 'Female  '],
      dtype=object)

In [12]:
tidy.drop(tidy[tidy.Sex.isin(['% of Total', '% of all Males ','% of all Females'])].index, inplace = True)

In [13]:
tidy['Sex'].unique()

array(['Persons', 'Male', 'Female', 'Male (%)', 'Female (%)', 'Female  '],
      dtype=object)

In [14]:
tidy['Period'].unique()

array(['1 March 2017', '1st March 2007', '1st March 2010',
       '1st March 2012', '1st September2014'], dtype=object)

In [15]:
from datetime import datetime

In [16]:
tidy.Period = pd.to_datetime(tidy.Period).dt.strftime('%Y-%m-%d')

In [17]:
tidy['Period'] = str('day/') + tidy['Period']

In [18]:
tidy['Age'].unique()

array(['Under 18 ', '18 and over', 'All years', 'All'], dtype=object)

In [19]:
tidy['Age'] = tidy['Age'].map(
    lambda x: {
        'Under 18 ' : 'under-18', 
        '18 and over' : '18-plus',
        'All years': 'all' ,
        'All': 'all'
        }.get(x, x))

In [20]:
tidy['Treatment Type'] = 'alcohol'

In [21]:
def user_perc(x,y):
    
    if ( (str(x) == 'Statutory (%)')) | ((str(x) == 'Non-statutory (%)')) | ((str(x) == 'Prison (%)')): 
        
        return 'Percentage'
    else:
        return y
    
tidy['Measure Type'] = tidy.apply(lambda row: user_perc(row['Service Type'], row['Measure Type']), axis = 1)



In [22]:
def user_perc(x,y):
    
    if ( (str(x) == 'Residential (%)')) | ((str(x) == 'Non-residential (%)')) | ((str(x) == 'Mixed (%)')): 
        
        return 'Percentage'
    else:
        return y
    
tidy['Measure Type'] = tidy.apply(lambda row: user_perc(row['Residential Status'], row['Measure Type']), axis = 1)



In [23]:
tidy['Health and Social Care Trust'].unique()

array(['All', 'Belfast', 'Northern', 'South Eastern', 'Southern',
       'Western', 'Northern (%)', 'South Eastern (%)', 'Southern (%)',
       'Western (%)', 'Emergency admissions (HIS) (%)',
       'Emergency admissions (HIS)', 'Belfast (%)',
       'Emergency admissions (HIS)  (%)'], dtype=object)

In [24]:
def user_perc(x,y):
    
    if ( (str(x) == 'Belfast (%)')) | ((str(x) == 'Northern (%)')) | ((str(x) == 'South Eastern (%)')) | ((str(x) == 'Southern (%)')) | ((str(x) == 'Western (%)')) | ((str(x) == 'Emergency admissions (HIS) (%)')) : 
        
            return 'Percentage'
    else:
            return y
    
tidy['Measure Type'] = tidy.apply(lambda row: user_perc(row['Health and Social Care Trust'], row['Measure Type']), axis = 1)

In [25]:
def user_perc(x,y):
    
    if  ((str(x) == 'Emergency admissions (HIS)  (%)')) : 
        
            return 'Percentage'
    else:
            return y
    
tidy['Measure Type'] = tidy.apply(lambda row: user_perc(row['Health and Social Care Trust'], row['Measure Type']), axis = 1)

In [26]:
tidy['Health and Social Care Trust'] = tidy['Health and Social Care Trust'].str.rstrip(' (%)')

In [27]:
tidy['Health and Social Care Trust'] = tidy['Health and Social Care Trust'].str.lower()

In [28]:
tidy['Health and Social Care Trust'].unique()

array(['all', 'belfast', 'northern', 'south eastern', 'southern',
       'western', 'emergency admissions (his'], dtype=object)

In [29]:
tidy['Health and Social Care Trust'] = tidy['Health and Social Care Trust'].map(
    lambda x: {
        'south eastern' : 'south-eastern', 
        'emergency admissions (his)' : 'emergency-admissions',
        'emergency admissions (his' : 'emergency-admissions'
        }.get(x, x))

In [30]:
tidy['Measure Type'] = tidy['Measure Type'].map(
    lambda x: {
        'Headcount' : 'Count', 
        'Percentage of Headcount' : 'Percentage',
        }.get(x, x))

In [31]:
tidy['Sex'].unique()

array(['Persons', 'Male', 'Female', 'Male (%)', 'Female (%)', 'Female  '],
      dtype=object)

In [32]:
def user_perc(x,y):
    
    if ( (str(x) == 'Male (%)')) | ((str(x) == 'Female (%)')): 
        
        return 'Percentage'
    else:
        return y
    
tidy['Measure Type'] = tidy.apply(lambda row: user_perc(row['Sex'], row['Measure Type']), axis = 1)



In [33]:
tidy['Sex'] = tidy['Sex'].map(
    lambda x: {
        'Female' : 'F', 
        'Male' : 'M',
        'Persons' : 'T',
        'Female  ' : 'F',
        'Male (%)' : 'M',
        'Female (%)': 'F'
        }.get(x, x))

In [34]:
tidy['Sex'] = tidy['Sex'].str.rstrip('(%)')

In [35]:
tidy['Service Type'].unique()

array(['All', 'Statutory', 'Non-statutory', 'Prison', 'Statutory (%)',
       'Non-statutory (%)', 'Prison (%)'], dtype=object)

In [36]:
tidy['Service Type'] = tidy['Service Type'].map(
    lambda x: {
        'Total' : 'all',
        'total' : 'all'
        }.get(x, x))

In [37]:
tidy['Service Type'] = tidy['Service Type'].str.rstrip(' (%)')

In [38]:
tidy['Service Type'] = tidy['Service Type'].str.lower()

In [39]:
tidy['Residential Status'] = tidy['Residential Status'].str.rstrip(' (%)')

In [40]:
tidy['Residential Status'] = tidy['Residential Status'].str.lower()

In [41]:
tidy['Health and Social Care Trust'] = tidy['Health and Social Care Trust'].map(
    lambda x: {
        'total' : 'all' 
        }.get(x, x))

In [42]:
tidy['Residential Status'] = tidy['Residential Status'].map(
    lambda x: {
        'total' : 'all' 
        }.get(x, x))

In [43]:
tidy.head()

,Period,Sex,Age,Service Type,Residential Status,Treatment Type,Health and Social Care Trust,Measure Type,Unit,Value
0,day/2017-03-01,T,under-18,all,all,alcohol,all,Count,People,95
4,day/2017-03-01,T,18-plus,all,all,alcohol,all,Count,People,2482
8,day/2017-03-01,T,all,all,all,alcohol,all,Count,People,2577
12,day/2017-03-01,M,under-18,all,all,alcohol,all,Count,People,31
16,day/2017-03-01,M,18-plus,all,all,alcohol,all,Count,People,1536


In [44]:
from pathlib import Path
out = Path('out')
out.mkdir(exist_ok=True)
tidy.to_csv(out / 'observations.csv', index = False)

In [45]:
tidy[tidy['Measure Type'] != 'Percentage'].to_csv(out / 'observations-no-percentages.csv')

There's a metadata tab in the spreadsheet with abstract and contact details. **Todo: extract these and also figure out what the license should really be.**

In [46]:
md_tab = tabs['Metadata']
md_heading = md_tab.filter('Metadata')
md_heading.assert_one()
properties = md_heading.fill(DOWN).is_not_whitespace()
values = properties.fill(RIGHT).is_not_whitespace()
headings = properties - values.fill(LEFT)
properties = properties - headings
cs = ConversionSegment(values, [
    HDim(properties, "Property", DIRECTLY, LEFT),
    HDim(headings, "Heading", CLOSEST, UP)
], includecellxy=True)
savepreviewhtml(cs)

In [47]:
from gssutils.metadata import THEME

for i, row in cs.topandas().iterrows():
    v = md_tab.get_at(row.__x, row.__y).value.strip()
    if row.Property == 'Abstract':
        scraper.dataset.description = v
    elif row.Property == 'National Statistics Theme':
        scraper.dataset.theme = THEME[pathify(v)]
    elif row.Property == 'Keyword':
        scraper.dataset.keyword = map(lambda x: x.strip(), v.split(' '))
    elif row.Property == 'Email Address':
        scraper.dataset.contactEmail = f'mailto:{v}'

In [48]:
scraper.dataset.family = 'health'
scraper.dataset.license = 'http://www.nationalarchives.gov.uk/doc/open-government-licence/version/3/'

with open(out / 'dataset.trig', 'wb') as metadata:
    metadata.write(scraper.generate_trig())